<a href="https://colab.research.google.com/github/FrederikKober/Thesis/blob/main/Rolling_App1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Reduced script that covers:
    --> rolling implementation for:
        
        i) Simple Neural Network with forward rates
        ii) Simple LSTM-Model with forward rates
"""


# Imports
import numpy as np
import pandas as pd
import scipy.io as sio
import multiprocessing as mp
import os
import time
import statsmodels.api as sm
from scipy.stats import t as tstat
import shutil

import NN_Func_App1 as NFB


# User Functions


def multProcessOwnExog(NNfunc, ncpus, _nMC, X, Xexog, Y, **kwargs):
    try:
        pool = mp.Pool(processes=ncpus)
        output = [pool.apply_async(NNfunc, args=(X, Xexog, Y, no,),
                                   kwds=kwargs)
                  for no in range(_nMC)]
        outputCons = [p.get(timeout=3000) for p in output]
        pool.close()
        pool.join()
        time.sleep(1)

    except Exception as e:
        print(e)
        print("Timed out, shutting pool down")
        pool.close()
        pool.terminate()
        time.sleep(1)

    return outputCons


def R2OOS(y_true, y_forecast):
    import numpy as np

    # Compute conidtional mean forecast
    y_condmean = np.divide(y_true.cumsum(), (np.arange(y_true.size)+1))

    # lag by one period
    y_condmean = np.insert(y_condmean, 0, np.nan)
    y_condmean = y_condmean[:-1] 
    y_condmean[np.isnan(y_forecast)] = np.nan 

    # Sum of Squared Resids
    SSres = np.nansum(np.square(y_true-y_forecast))
    SStot = np.nansum(np.square(y_true-y_condmean))

    return 1-SSres/SStot


def RSZ_Signif(y_true, y_forecast):

    # Compute conidtional mean forecast
    y_condmean = np.divide(y_true.cumsum(), (np.arange(y_true.size)+1))

    # lag by one period
    y_condmean = np.insert(y_condmean, 0, np.nan)
    y_condmean = y_condmean[:-1]
    # y_condmean[np.isnan(y_forecast)] = np.nan # ! NOt needed here in update

    # Compute f-measure
        # !!! According to paper Rpach Strauss... page 8-9 the last + should be -
    f = np.square(y_true-y_condmean)-np.square(y_true-y_forecast)  \
        + np.square(y_condmean-y_forecast)

    # Regress f on a constant
    x = np.ones(np.shape(f))
    model = sm.OLS(f, x, missing='drop', hasconst=True)
    results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 12})

    return 1-tstat.cdf(results.tvalues[0], results.nobs-1)


if __name__ == "__main__":

    # =========================================================================
    #                           Settings
    # =========================================================================

    # Set True for testing, False otherwise. Reduces computational complexity.
    # TestFlag is for development purposes only.
    TestFlag = True
    if TestFlag:
        _nMC = 10  # Number of networks to train in parallel
        _nAvg = 6  # Number of best networks to take for model averaging
    else:
        _nMC = 100
        _nAvg = 10

    OOS_Start = '1989-02-01'  # Date of RHS variable where to start OOS

    HyperFreq = 2*24  # Frequency of hyper-parameter search # !!! DECREASED
    NN_Params = {'Dropout': [0.1, 0.3, 0.5], 'l1l2': [0.01, 0.001]}
    
    # For LSTM
    other_gen = {"_len": 12, "_batch_top": 32, "_batch_bot": 32}

    # Computational Ressources: Determine Number of available cores
    ncpus = mp.cpu_count()
    print("CPU count is: "+str(ncpus))

    # Location for temporary model storage. Deleted upon completion.
    dumploc_base = './trainingDumps_'

    i = 0
    path_established = False
    while not path_established:
        dumploc = dumploc_base+str(i)
        try:
            os.mkdir(dumploc)
            print("Directory ", dumploc, " Created ")
            path_established = True
        except FileExistsError:
            print("Directory ", dumploc, " Already exists")
            i += 1

    # Set of models for training
    #models = [NFB.NN3LayerExog, NFB.ElasticNet_Exog_Plain, NFB.NN1LayerEnsemExog]
    #models = [NFB.NNForwardRun, NFB.NN3LayerExog, NFB.NN1LayerEnsemExog_2D, NFB.NN1LayerEnsemExog_3D]
    #models = [NFB.NNForwardRun , NFB.NN1LayerEnsemExog_2D]
    #models =  [NFB.NN3LayerExog, NFB.NNForwardRun, NFB.NN1LayerEnsemExog_2D]
    #models = [NFB.NNForwardRun]
    # Set of names to use for models. Same order as in list "models".
    #modelnames = ["NNForwardRun", "NN3LayerExog", "NN1LayerEnsemExog_2D", "NN1LayerEnsemExog_3D"]
    #modelnames = ["NNForwardRun", "NN1LayerEnsemExog_2D"]
    #modelnames = ["NN3LayerExog", "NNForwardFunction", "NN1LayerEnsemExog_2D"]
    #modelnames = ["NNForwardFunction"]
    
    #acrch_all = {"NNForwardFunction": [[32,16,8],[16,8],[8]]}
    #arch_all = [[32,16,8],[16,8],[8]]
    #archi = [32,16,8]

    # =========================================================================
    #                          Data Loading
    # =========================================================================


    ###############
    ## Load in Data
    
    # Load data from github repository
    _url = "https://github.com/FrederikKober/Thesis/blob/700edaddeea6644c26bacaa31227762663385652/dataset1_2.xlsx?raw=true"

    # for index
    #_idx = pd.read_excel("/home/freddy/Dropbox/Thesis_Final/prepared_data/dataset1_2.xlsx", sheet_name="dataset1", index_col=0)
    _idx = pd.read_excel(_url, sheet_name="dataset1", index_col=0)

    # Excess Returns (Y) --> TxM --> T observations with M variables forecasted simultaneously
    Y = np.array(_idx.iloc[:,:9]) 
    
    # Forward rates (TxN) --> with N number of forward rates
    Xexog = np.array(_idx.iloc[:,9:])
    
    # Macro data 
    #X = pd.read_excel("/home/freddy/Dropbox/Thesis_Final/prepared_data/dataset1_2.xlsx", sheet_name="dataset2", index_col=0)
    X = pd.read_excel(_url, sheet_name="dataset2", index_col=0)
    _rpi = np.where(X.columns == "RPI")[0][0]
    X = np.array(X.iloc[:,_rpi:])


    #######################
    ## Define Architectures
    model_dict = {"Run_LSTM_Generic": [NFB.Run_LSTM_Generic, [[7,4],[5,2]], ["_7_4","_5_2"]],
                  "NNForwardFunction": [NFB.NNForwardRun, [[16,8],[8,4],[8]], ["_16_8", "_8_4", "_8"]]}
    
    
    # =========================================================================
    #                   Estimation
    # =========================================================================


    # Determine IS vs OS indices
    _T = Xexog.shape[0]
    tstart = np.argmax(_idx.index == OOS_Start)
    OoS_indeces = range(tstart, int(_T))
    # Number of outputs
    _M = Y.shape[1]

    if TestFlag:
        OoS_indeces = OoS_indeces[:10]

    VarSave = {}  # Storage dictionary for all results

    # Loop over models
    #for modelnum, modelfunc in enumerate(models): ## !! modelnum, modelfunc --> needs to spit out  names and functions
    for function in model_dict:
    
        Y_forecast = np.full([_T, _nMC, _M], np.nan)
        Y_forecast_agg = np.full([_T, _M], np.nan)
        val_loss = np.full([_T, _nMC], np.nan)
        #print(modelnames[modelnum]) # !! modelnum and modelnames


        
        #====================================================================#
        # Model Case Distinction
        #====================================================================#
        
        #######################
        ## NNForwardFunction ##
        
        if function == 'NNForwardFunction': 
            # Loop over Model architectures 
            for _num, _arch in enumerate(model_dict[function][1]):
                # Define
                archi = _arch
                modelname = function + model_dict[function][2][_num]
                modelfunc = model_dict[function][0]
                print(archi)
            
                j = 1
                for i in OoS_indeces:
    
                    # Determine whether to perform fresh hyper-parameter search
                    if (j == 1) or (j % HyperFreq == 0):
                        refit = True
                    else:
                        refit = False
                    j += 1
    
                    # Run model
                    start = time.time()
                    output = multProcessOwnExog(modelfunc, ncpus, _nMC, X[:i+1, :], 
                                                Xexog[:i+1, :], Y[:i+1, :],
                                                dumploc=dumploc, params=NN_Params,
                                                refit=refit, archi = archi)
                    # Handle output
                    val_loss[i, :] = np.array([output[k][1] for k in range(_nMC)])
                    Y_forecast[i, :, :] = np.concatenate([output[k][0] for k
                                                          in range(_nMC)], axis=0)
                    tempsort = np.argsort(val_loss[i, :])
                    ypredmean = np.mean(Y_forecast[i, tempsort[:_nAvg], :], axis=0)
                    Y_forecast_agg[i, :] = ypredmean
                    print("Obs No.: ", i, " - Step Time: ", time.time() - start)

                # Validation Loss
                VarSave["ValLoss_"+modelname] = val_loss # !! modelnames[modelnum]
                
                # Fitted / Forecasted Values (Averaged)
                VarSave["Y_forecast_agg_"+modelname] = Y_forecast_agg #  !! modelnames[modelnum]
        
                # Fitted / Forecasted Values (From models trained in parallel)
                VarSave["Y_forecast_"+modelname] = Y_forecast # !! modelnames[modelnum]
        
                # Mean Squared Error
                VarSave["MSE_"+modelname] = np.nanmean(np.square(Y-Y_forecast_agg), axis=0)  # !! modelnames[modelnum]
        
                # R2-00S
                VarSave["R2OOS_"+modelname] = np.array( # !! modelnames[modelnum]
                    [R2OOS(Y[:, k], Y_forecast_agg[:, k]) for k in range(np.size(Y, axis=1))])
        
                print('R2OOS: ', VarSave["R2OOS_"+modelname]) # !! modelnames[modelnum]
        
                # Rapach, Strauss, Zhou (2010) - Significance R^2
                VarSave["R2OOS_pval_"+modelname] = \
                    np.array([RSZ_Signif(Y[:, k], Y_forecast_agg[:, k])
                              for k in range(np.size(Y, axis=1))])
          

        ######################################################################
        ######################################################################
        
        ## Run_LSTM_Generic ##
        
        elif function == 'Run_LSTM_Generic': 
            # Defined above
            #other = {"_len": 3, "_batch_top": 32, "_batch_bot": 32}

       
            # Loop over Model architectures 
            for _num, _arch in enumerate(model_dict[function][1]):
                # Define
                archi = _arch
                modelname = function + model_dict[function][2][_num]
                modelfunc = model_dict[function][0]
                print(archi)
            
                j = 1
                for i in OoS_indeces:
    
                    # Determine whether to perform fresh hyper-parameter search
                    if (j == 1) or (j % HyperFreq == 0):
                        refit = True
                    else:
                        refit = False
                    j += 1
    
                    # Run model
                    start = time.time()
                    output = multProcessOwnExog(modelfunc, ncpus, _nMC, X[:i+1, :], 
                                                Xexog[:i+1, :], Y[:i+1, :],
                                                dumploc=dumploc, params=NN_Params,
                                                refit=refit, archi = archi, other_gen = other_gen) # !!! HERE
                    # Handle output
                    val_loss[i, :] = np.array([output[k][1] for k in range(_nMC)])
                    Y_forecast[i, :, :] = np.concatenate([output[k][0] for k
                                                          in range(_nMC)], axis=0)
                    tempsort = np.argsort(val_loss[i, :])
                    ypredmean = np.mean(Y_forecast[i, tempsort[:_nAvg], :], axis=0)
                    Y_forecast_agg[i, :] = ypredmean
                    print("Obs No.: ", i, " - Step Time: ", time.time() - start)

                # Validation Loss
                VarSave["ValLoss_"+modelname] = val_loss # !! modelnames[modelnum]
                
                # Fitted / Forecasted Values (Averaged)
                VarSave["Y_forecast_agg_"+modelname] = Y_forecast_agg #  !! modelnames[modelnum]
        
                # Fitted / Forecasted Values (From models trained in parallel)
                VarSave["Y_forecast_"+modelname] = Y_forecast # !! modelnames[modelnum]
        
                # Mean Squared Error
                VarSave["MSE_"+modelname] = np.nanmean(np.square(Y-Y_forecast_agg), axis=0)  # !! modelnames[modelnum]
        
                # R2-00S
                VarSave["R2OOS_"+modelname] = np.array( # !! modelnames[modelnum]
                    [R2OOS(Y[:, k], Y_forecast_agg[:, k]) for k in range(np.size(Y, axis=1))])
        
                print('R2OOS: ', VarSave["R2OOS_"+modelname]) # !! modelnames[modelnum]
        
                # Rapach, Strauss, Zhou (2010) - Significance R^2
                VarSave["R2OOS_pval_"+modelname] = \
                    np.array([RSZ_Signif(Y[:, k], Y_forecast_agg[:, k])
                              for k in range(np.size(Y, axis=1))])
                                 
                
        else:
            raise Exception("Model does not match any known case.")


        savesuccess_flag = False
        while not savesuccess_flag:
            # Save new result to SOA file
            try:
                VarSaveSOA = sio.loadmat('ModelComparison_Rolling_SOA.mat')
                VarSaveSOA.update(VarSave)
                sio.savemat('ModelComparison_Rolling_SOA.mat', VarSaveSOA)
                savesuccess_flag = True
                print('Updated SOA file')
            except FileNotFoundError:
                sio.savemat('ModelComparison_Rolling_SOA.mat', VarSave)
                savesuccess_flag = True
                print('Created new SOA file')

    # Delete Dumploc
    try:
        shutil.rmtree(dumploc)
        print('Removed dir: '+dumploc+' succesfully')
    except FileNotFoundError:
        print('Directory: '+dumploc+' could not be removed')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


ModuleNotFoundError: ignored